In [42]:
import pandas as pd
import geopandas as gpd

data_path = '/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/'

In [43]:
#get BC regional district average incomes and get deciles
#from https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/download-telecharger.cfm?Lang=E

# BC Variables

In [44]:
BC_data = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/census/Canada/BC_income_v2.CSV', encoding='latin-1')
mob_vars = pd.read_csv(data_path + 'mobility_heat/mobility_heat_vars_clean_90.csv')


In [ ]:
lst = [x for x in BC_data['GEO_NAME'] if 'Regional district' in x]
BC_data = BC_data[BC_data['GEO_NAME'].isin(lst)]


household_income = BC_data[BC_data['CHARACTERISTIC_NAME'] == '  Median total income of household in 2020 ($)']
household_income['median_income'] = household_income['C1_COUNT_TOTAL']

population = BC_data[BC_data['CHARACTERISTIC_NAME'] == 'Population, 2021']
population['population'] = population['C1_COUNT_TOTAL']

age = BC_data[BC_data['CHARACTERISTIC_NAME'] == '  65 years and over']
age['65_and_over'] = age['C1_COUNT_TOTAL']

nonwhite = BC_data[BC_data['CHARACTERISTIC_NAME'] == '  Total visible minority population']
nonwhite['nonwhite_pop'] = nonwhite['C1_COUNT_TOTAL']



In [46]:
us_areas = pd.read_csv(data_path + 'census/US/pop_dens.csv')
ca_areas = pd.read_html("https://en.wikipedia.org/wiki/List_of_regional_districts_of_British_Columbia")

us_areas = us_areas[['NAME', 'State', 'B01001_calc_PopDensity']]

ca_areas = ca_areas[1][['Regional district', 'Density (per km2)']]
ca_areas.replace({'–': '-'}, regex=True, inplace=True)
ca_areas.replace('Metro Vancouver', 'Greater Vancouver', inplace=True)

us_areas.columns = ['county', 'state', 'popdens']

ca_areas.columns = ['county', 'popdens']
ca_areas['state'] = 'British Columbia'

pop_dens = pd.concat([us_areas, ca_areas])

pop_dens['location'] = pop_dens['county'] + ', ' + pop_dens['state']

In [47]:
pop_dens = pop_dens[pop_dens['state'].isin(['British Columbia', 'California', 'Oregon', 'Washington'])]

In [48]:
pop_dens['popdens_dec'] = pop_dens.groupby('state')['popdens'].transform(lambda x: pd.qcut(x, 10, labels=False))
pop_dens['popdens_dec'] += 1

In [50]:
age = age.groupby('DGUID').first().reset_index()

In [51]:
income_pop = household_income.merge(population[['DGUID', 'CHARACTERISTIC_NAME', 'population']], on='DGUID')

In [52]:
income_pop_age = income_pop.merge(age[['DGUID', 'CHARACTERISTIC_NAME', '65_and_over']], on='DGUID')

In [53]:
can_total = income_pop_age.merge(nonwhite[['DGUID', 'CHARACTERISTIC_NAME', 'nonwhite_pop']], on='DGUID')

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_97576/2573391190.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'CHARACTERISTIC_NAME_x'} in the result is deprecated and will raise a MergeError in a future version.
  can_total = income_pop_age.merge(nonwhite[['DGUID', 'CHARACTERISTIC_NAME', 'nonwhite_pop']], on='DGUID')


In [54]:
can_total['district'] = [x.split(',')[0] for x in can_total['GEO_NAME']]

In [55]:
can_total['propor_over_65'] = can_total['65_and_over'] / can_total['population']

In [56]:
can_total['non_white_propor'] = can_total['nonwhite_pop'] / can_total['population']

In [57]:
can_total['income_dec'] = pd.qcut(can_total['median_income'],10,labels=False)
can_total['income_dec'] += 1

In [58]:
can_total['over65_dec'] = pd.qcut(can_total['propor_over_65'],10,labels=False)
can_total['over65_dec'] += 1

In [59]:
can_total['nonwhite_dec'] = pd.qcut(can_total['non_white_propor'],10,labels=False)
can_total['nonwhite_dec'] += 1

In [63]:
bc_mob_vars = mob_vars.merge(can_total[['district', 'income_dec',  'over65_dec', 'nonwhite_dec']], left_on='county', right_on='district')

In [64]:
bc_mob_vars.columns

Index(['Unnamed: 0.2', 'index', 'Unnamed: 0.1', 'Unnamed: 0',
       'country_region_code', 'country_region', 'state', 'county',
       'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id',
       'date_heat', 'retail_heat', 'grocery_pharmacy_heat', 'parks_heat',
       'transit_heat', 'workplaces_heat', 'residential_heat', 'year',
       'location', 'geometry', 'control_day', 'temperature_2m_heat',
       'heat_ID_x', 'date_control', 'retail_control',
       'grocery_pharmacy_control', 'parks_control', 'transit_control',
       'workplaces_control', 'residential_control', 'retail_difference',
       'grocery_pharmacy_difference', 'parks_difference', 'transit_difference',
       'workplaces_difference', 'residential_difference', 'day_of_week',
       'weekend', 'holiday', 'us_holiday_heat', 'us_holiday', 'C',
       'datetime_heat', 'temperature_2m_control', 'datetime_control',
       'temperature_2m', 'temp_diff', 'heat_ID_y', 'date',
       'Heat Wave Duration', 'dur_ranki

# U.S. Variables

In [66]:
mob_vars = pd.read_csv(data_path + 'mobility_heat/mobility_heat_vars_clean_90.csv')
#mob_vars.drop(['over65_dec', 'income_dec'],axis=1, inplace=True)


us_age = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/census/US/plus65.csv', encoding='latin-1')
us_income = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/census/US/county_income.csv', encoding='latin-1')
us_race = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/census/US/county_white.csv', encoding='latin-1')

In [67]:
total_pop = us_age[us_age['variable'] == 'total_pop']
total_pop['total_pop'] = total_pop['estimate']

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_97576/3177610766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_pop['total_pop'] = total_pop['estimate']


In [68]:
us_age = us_age[us_age['variable'] != 'total_pop']

In [69]:
us_age = us_age.groupby('NAME').sum().reset_index()

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_97576/1343126567.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  us_age = us_age.groupby('NAME').sum().reset_index()


In [70]:
us_age['total65plus'] = us_age['estimate']

In [71]:
us_age = us_age.merge(total_pop, on="NAME")

In [72]:
us_age['propor_over_65'] = us_age['total65plus'] / us_age['total_pop']

In [73]:
us_income['median_income'] = us_income['estimate']

In [74]:
us_income_age = us_age.merge(us_income, on='NAME')

In [75]:
us_income_age['state'] = [x.split(',')[-1].strip() for x in us_income_age['NAME']]

In [76]:
us_income_age = us_income_age[us_income_age['state'].isin(['California', 'Washington', 'Oregon'])]

In [78]:
us_race['white_pop'] = us_race['estimate']

In [79]:
us_race = us_race.merge(total_pop, on="NAME")

In [80]:
us_race['non_white_propor'] = 1 - (us_race['white_pop'] / us_race['total_pop'])

In [82]:
us_total = us_income_age.merge(us_race, on='NAME')

In [83]:
us_total['state'] = [x.split(',')[-1].strip() for x in us_total['NAME']]

In [84]:
us_total = us_total[us_total['state'].isin(['California', 'Washington', 'Oregon'])]

In [85]:
us_total = us_total[['NAME', 'propor_over_65', 'median_income', 'state', 'non_white_propor']]

In [86]:
us_total['income_dec'] = us_total.groupby('state')['median_income'].transform(lambda x: pd.qcut(x, 10, labels=False))
us_total['income_dec'] += 1

In [87]:
us_total['over65_dec'] = us_total.groupby('state')['propor_over_65'].transform(lambda x: pd.qcut(x, 10, labels=False))
us_total['over65_dec'] += 1

In [88]:
us_total['nonwhite_dec'] = us_total.groupby('state')['non_white_propor'].transform(lambda x: pd.qcut(x, 10, labels=False))
us_total['nonwhite_dec'] += 1

In [89]:
mob_vars = mob_vars.merge(us_total[['NAME', 'over65_dec', 'income_dec', 'nonwhite_dec']], left_on='location', right_on='NAME')

In [90]:
mob_vars['over65_dec'].min()

1

# Merge BC and U.S. and save

In [91]:
mob_vars = mob_vars.append(bc_mob_vars)

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_97576/2836113146.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mob_vars = mob_vars.append(bc_mob_vars)


In [93]:
mob_vars.drop('district', axis=1, inplace=True)

In [94]:
mob_vars = mob_vars.merge(pop_dens[['location', 'popdens_dec']], on='location')

In [100]:
mob_vars.to_csv(data_path + 'mobility_heat/mob_vars_final_clean_90.csv')